In [ ]:
import tensorflow as tf
from tensorflow import keras
from nltk.stem import WordNetLemmatizer 
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import nltk
import enchant
import pickle
import numpy as np
from nltk.corpus import wordnet, stopwords
from nltk.stem import PorterStemmer 
from nltk.lm import Vocabulary
from nltk.tag import StanfordNERTagger, pos_tag
from sklearn.model_selection import learning_curve
import glob
import contractions
import string
import bs4 as bs
import pandas as pd
from collections import Counter
import urllib.request
import re
import multiprocessing
import ndjson
from gensim.models import Word2Vec
import ssl
import getch
from google.cloud import storage
import spacy
from spacy import displacy
import en_core_web_sm
nlp = en_core_web_sm.load()

from sklearn.utils import shuffle
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from gensim.models import KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec

from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.models import model_from_json, Sequential
from tensorflow.python.keras.layers import Dense, Dropout, Embedding, Flatten, Activation, Bidirectional, LSTM, GRU
from tensorflow.python.keras.optimizers import Adam
from tensorflow.python.keras.callbacks import EarlyStopping
from tensorflow.python.keras.metrics import categorical_accuracy
from tensorflow.python.keras.utils import to_categorical
from tensorflow.python.keras import backend as K


%load_ext autotime
import matplotlib.pyplot as plt
%matplotlib inline

np.random.seed(7)

import warnings
warnings.filterwarnings('ignore')

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

import os

savepath = '/Users/animallya/Desktop/NLP_project/project/SeekGen_flask_app/SeekGen/savefiles/'
embedpath = '/Users/animallya/Desktop/NLP_project/project/SeekGen_flask_app/SeekGen/embeddings/'

ngram_size = 3

In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

In [ ]:
import sqlite3
import pandas as pd

lemmatizer = WordNetLemmatizer() 

def filter_words(text, global_vocab):
    words = []
    for w in text:
        w = w.lower()
        if len(w)>1:
            if global_vocab.check(w)==True:            
                if '\'' in w or '’' in w:
                    if contractions.fix(w) != w:
                        fixedw = contractions.fix(w).lower()
                        words.append(fixedw)
                else:
                    words.append(w)
        else:
            if w in['i','a'] or w in string.punctuation:
                words.append(w)
    return ' '.join(words)

def message_scrape(path, ids):
    
    conn = sqlite3.connect(path)
    c = conn.cursor()
    output = ""
    
    for n in range(ids):
        cmd1 = 'SELECT ROWID, text, handle_id, \
                    datetime(date + strftime(\'%s\',\'2001-01-01\'), \'unixepoch\') as date_utc \
                    FROM message T1 \
                    INNER JOIN chat_message_join T2 \
                        ON T2.chat_id='+str(n)+ ' \
                        AND T1.ROWID=T2.message_id \
                    ORDER BY T1.date'
        c.execute(cmd1)
        df_msg = pd.DataFrame(c.fetchall(), columns=['id', 'text', 'sender', 'time'])
        corpus = df_msg.text.tolist()
        output +=' '+' '.join(corpus)

    corpus = re.sub('[^a-zA-Z\'\’\?]', ' ', output)
    corpus = re.sub(r'\s+', ' ', corpus)
    corpus = corpus.lower().split(" ")
    filtered_corpus = filter_words(corpus, enchant.Dict("en_US"))
    '''
    frequency filtering, use nltk library to get frequency of all unique tokens in corpus and remove all of those that are
    above some threshold.(research what threshold you wana use?)
    When you lemmatize a word, store its original form as the value to the root form of the word. Root form is the key in a
    dictionary. Ex: {'go':'went','going'} and then we can use that dictionary to de-lemmatize it during prediction.
    '''
    return filtered_corpus

## Trigram Neural Model predictor

In [ ]:
# generate glove embeddings savefile, uncomment next 3 lines if running for first time.
# glove_input_file = embedpath+'glove.42B.300d.txt'
word2vec_output_file = embedpath+'glove.42B.300d.txt.word2vec'
# glove2word2vec(glove_input_file, word2vec_output_file)
embeddings = KeyedVectors.load_word2vec_format(word2vec_output_file, binary=False)

In [ ]:
def preprocess():
    corpus = message_scrape(path=savepath+'chat1.db',ids=100)+' '+ message_scrape(path=savepath+'chat2.db',ids=100)
    words = nltk.word_tokenize(corpus)
    #automate this later
    words = list(filter(lambda a: a != 'bet', words))
    vocab = list(Vocabulary(words))
    return vocab, words, corpus

def generate_traindata(vocab, words):
    #Y = [vocab.index(words[i]) for i in range(ngram_size, len(words))]
    Y = [embeddings[word] for i in range(ngram_size, len(words))]
    X = [np.array([embeddings[word] for word in words[i-ngram_size:i]]) for i in range(ngram_size, len(words))]
    Y = [to_categorical(y,len(vocab)) for y in Y]
    X, Y = shuffle(X, Y, random_state=42)
    return np.array(X),np.array(Y)

In [ ]:
vocab, words, corpus = preprocess()

In [ ]:
len(vocab)

In [ ]:
X,y = generate_traindata(vocab, words)

In [ ]:
print(X.shape, y.shape)

In [ ]:
X[0]

In [ ]:
y[0]

In [ ]:
def bidirectional_lstm_model(X):
    inputs = keras.Input(shape=(X.shape[1], X.shape[2]), name='digits')
    x = keras.Sequential()(inputs)
    x = Bidirectional(LSTM(512,activation="tanh"))(x)
    outputs = Dense(len(vocab),activation='softmax')(x)
    model = keras.Model(inputs=inputs, outputs=outputs, name='bdrnn')
    print("model built!")
    return model

In [ ]:
rnn_model = bidirectional_lstm_model(X)
rnn_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=[categorical_accuracy])

In [ ]:
history_rnn = rnn_model.fit(X, y, batch_size=128, epochs= 25, validation_split = 0.1)

In [ ]:
plt.plot(history_rnn.history['val_categorical_accuracy'])
plt.title('validation loss')
plt.ylabel('val loss')
plt.xlabel('Epoch')
plt.show()

## Pos-tagging using SVC model

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

file = nltk.corpus.webtext.fileids()[-1] #ID is the last one
#webtext = ' '.join(nltk.corpus.reuters.words(file))
doc = nlp(corpus)
label_encoder = LabelEncoder()

xtemp = [token.tag_ for token in doc]
encoded = label_encoder.fit_transform(xtemp)
X_encoded = [encoded[i-ngram_size:i] for i in range(ngram_size,len(encoded))]
y_encoded = [encoded[i] for i in range(ngram_size,len(encoded))]
y_encoded = np.array(y_encoded)
X_encoded = np.array(X_encoded)

In [ ]:
X_encoded.shape

In [ ]:
tag_model = SVC(gamma='auto',probability=True, class_weight='balanced',verbose=2)
tag_model.fit(X_encoded, y_encoded)

### Saving all 3 models

In [ ]:
save_path = '/Users/animallya/Desktop/NLP_project/project/SeekGen_flask_app/SeekGen/savefiles/'

In [ ]:
def save_model(model, model_name):
    model.save(save_path+model_name+'.h5') 

#save_model(rnn_model,'rnn_model')

# filename = save_path+'postagger.sav'
# pickle.dump(tag_model, open(filename, 'wb'))

# TESTING


In [ ]:
rnn_model = tf.keras.models.load_model(save_path+'rnn_model.h5')
tag_model = pickle.load(open(save_path+'postagger.sav', 'rb'))

In [ ]:
def input_tag_to_prediction(inp):
    doc = nlp(inp.lower())
    tags = [token.tag_ for token in doc]
    X_encoded = label_encoder.transform(tags)
    y_pred = tag_model.predict_proba(X_encoded.reshape(1, -1))[0]
    indices = (y_pred).argsort()[::-1][:3]
    return [label_encoder.inverse_transform([idx])[0] for idx in indices]

In [ ]:
def input_word_to_prediction(inp):
    words = nltk.word_tokenize(inp.lower())
    words_tf = np.array([embeddings[word] for word in words]).reshape(1, -1)
    y_pred = sequence_model.predict_proba(words_tf)[0]
    indices = (y_pred).argsort()[::-1][:3]
    return [vocab[idx] for idx in indices]

In [ ]:
def rnn_predict(sent):
    words = nltk.word_tokenize(sent.lower())
    x = np.array([embeddings[word] for word in words])
    words_tf = x.reshape(1,x.shape[0],x.shape[1])
    y_pred = np.array(rnn_model.predict(words_tf)[0])
    indices = (y_pred).argsort()[::-1][:3]
    return [vocab[idx] for idx in indices]

In [ ]:
def prediction_union(sent):
    tags = input_tag_to_prediction(sent)
    superset_tags = [tag[:2] for tag in tags]
    predictions = rnn_predict(sent)
    prediction_tags = [nlp(word)[0].tag_[:2] for word in predictions]
    filtered_predictions = [word for word in predictions if nlp(word)[0].tag_[:2] in superset_tags]
    return predictions, filtered_predictions

In [ ]:
def run(sent):
    out = sent
    while True:
        print(out)
        predictions, filtered_predictions = prediction_union(sent)
        print(predictions)
        print(filtered_predictions)
        inp = input()
        if inp=="exit":
            return out
        out+= " " +inp
        l = len(inp.split())
        sent = " ".join(nltk.word_tokenize(sent)[l:]+[inp])
    print(out)

In [ ]:
out = run("oh yeah what")
#i will be home by

In [ ]:
#enchant.Dict("en_US").suggest('sx')
embeddings.most_similar(positive=['king','female'], negative=['male'], topn=1)

In [ ]:
embeddings.most_similar('way')

In [ ]:
df_eval = pd.read_csv('SeekGen_flask_app/SeekGen/eval/evaluation.csv')
df_eval['Precision'] = df_eval['Pred_used'].divide(df_eval['Num_words'])
df_eval = df_eval.sort_values(by=['Precision'])

In [ ]:
df_eval.head(20)

In [ ]:
df_eval.tail()

In [ ]:
df_eval.Precision.mean()